## Installing required libraries

In [ ]:
import os

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

%pip install -Uq pip
%pip uninstall -q -y optimum optimum-intel
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu\
"llama-index" "faiss-cpu" "pymupdf" "llama-index-readers-file" "llama-index-vector-stores-faiss" "llama-index-llms-langchain" "llama-index-llms-openvino" "llama-index-embeddings-openvino" "llama-index-postprocessor-openvino-rerank" "transformers>=4.40" \
"git+https://github.com/huggingface/optimum-intel.git"\
"git+https://github.com/openvinotoolkit/nncf.git"\
"datasets"\
"accelerate"\
"gradio" \
"langchain"
%pip install --pre -Uq openvino openvino-tokenizers[transformers] --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
!pip install "llama-index" "faiss-cpu" "pymupdf" "llama-index-readers-file" "llama-index-vector-stores-faiss" "llama-index-llms-langchain" "llama-index-llms-openvino" "llama-index-embeddings-openvino" "llama-index-postprocessor-openvino-rerank" "transformers>=4.40" 

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 32.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 22.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 23.7 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 28.1 MB/s eta 0:00:

## Import required libraries and config

In [1]:
import os
from pathlib import Path
import requests
import shutil
import io

# fetch model configuration

config_shared_path = Path("../../utils/llm_config.py")
config_dst_path = Path("llm_config.py")

if not config_dst_path.exists():
    if config_shared_path.exists():
        try:
            os.symlink(config_shared_path, config_dst_path)
        except Exception:
            shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)
elif not os.path.islink(config_dst_path):
    print("LLM config will be updated")
    if config_shared_path.exists():
        shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)




LLM config will be updated


In [2]:
from pathlib import Path
import openvino as ov
import ipywidgets as widgets

In [3]:
from llm_config import (
    SUPPORTED_EMBEDDING_MODELS,
    SUPPORTED_RERANK_MODELS,
    SUPPORTED_LLM_MODELS,
)

model_language = "English" 

Dropdown(description='Model Language:', options=('English', 'Chinese', 'Japanese'), value='English')

In [5]:
llm_model_id = "gemma-2b-it"

Dropdown(description='Model:', index=12, options=('tiny-llama-1b-chat', 'gemma-2b-it', 'red-pajama-3b-chat', '…

## using openvino convert to openvino format and quantizing model to int8

In [7]:
def convert_to_int8():
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    int8_model_dir.mkdir(parents=True, exist_ok=True)
    remote_code = llm_model_configuration.get("remote_code", False)
    export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past --weight-format int8".format(pt_model_id)
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " " + str(int8_model_dir)
    display(Markdown("**Export command:**"))
    display(Markdown(f"`{export_command}`"))
    ! $export_command

convert_to_int8()

Checkbox(value=True, description='Prepare INT4 model')

Checkbox(value=False, description='Prepare INT8 model')

Checkbox(value=False, description='Prepare FP16 model')

Let's compare model size for different compression types


In [10]:
int8_weights = int8_model_dir / "openvino_model.bin"


for precision, compressed_weights in zip([8], [int8_weights]):
    if compressed_weights.exists():
        print(f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB")

Size of model with INT8 compressed weights is 2393.05 MB


## embedding model 

In [11]:
embedding_model_id = "bge-small-en-v1.5 model"

Dropdown(description='Embedding Model:', options=('bge-small-en-v1.5', 'bge-large-en-v1.5', 'bge-m3'), value='…

OpenVINO embedding model and tokenizer can be exported by `feature-extraction` task with `optimum-cli`.


In [13]:
export_command_base = "optimum-cli export openvino --model {} --task feature-extraction".format(embedding_model_configuration["model_id"])
export_command = export_command_base + " " + str(embedding_model_id.value)

if not Path(embedding_model_id.value).exists():
    ! $export_command

## Rerank model

In [14]:
rerank_model_id = "bge-reranker-large model"

Dropdown(description='Rerank Model:', options=('bge-reranker-v2-m3', 'bge-reranker-large', 'bge-reranker-base'…

In [18]:
export_command_base = "optimum-cli export openvino --model {} --task text-classification".format(rerank_model_configuration["model_id"])
export_command = export_command_base + " " + str(rerank_model_id.value)

if not Path(rerank_model_id.value).exists():
    ! $export_command

In [19]:
core = ov.Core()

support_devices = core.available_devices

embedding_device = widgets.Dropdown(
    options=support_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

embedding_device

Dropdown(description='Device:', options=('CPU', 'GPU', 'AUTO'), value='CPU')

In [20]:
print(f"Embedding model will be loaded to {embedding_device.value} device for text embedding")

Embedding model will be loaded to CPU device for text embedding


### Select device for rerank model inference

[back to top ⬆️](#Table-of-contents:)


In [21]:
rerank_device = widgets.Dropdown(
    options=support_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

rerank_device

Dropdown(description='Device:', options=('CPU', 'GPU', 'AUTO'), value='CPU')

In [22]:
print(f"Rerenk model will be loaded to {rerank_device.value} device for text reranking")

Rerenk model will be loaded to CPU device for text reranking


### Select device for LLM model inference

[back to top ⬆️](#Table-of-contents:)


In [23]:
llm_device = widgets.Dropdown(
    options=support_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

llm_device

Dropdown(description='Device:', options=('CPU', 'GPU', 'AUTO'), value='CPU')

In [24]:
print(f"LLM model will be loaded to {llm_device.value} device for response generation")

LLM model will be loaded to CPU device for response generation


## Load models

[back to top ⬆️](#Table-of-contents:)

### Load embedding model

[back to top ⬆️](#Table-of-contents:)

Now a Hugging Face embedding model can be supported by OpenVINO through [`OpenVINOEmbeddings`](https://docs.llamaindex.ai/en/stable/examples/embeddings/openvino/) class of LlamaIndex.


In [25]:
from llama_index.embeddings.huggingface_openvino import OpenVINOEmbedding


embedding = OpenVINOEmbedding(folder_name=embedding_model_id.value, device=embedding_device.value)

embeddings = embedding.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

2024-07-15 10:46:20.806937: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 10:46:21.548673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Compiling the model to CPU ...


384
[-0.0032756736036390066, -0.011690771207213402, 0.04155924171209335, -0.038148146122694016, 0.024183060973882675]


### Load rerank model

[back to top ⬆️](#Table-of-contents:)

Now a Hugging Face embedding model can be supported by OpenVINO through [`OpenVINORerank`](https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/openvino_rerank/) class of LlamaIndex.

> **Note**: Rerank can be skipped in RAG.


In [26]:
from llama_index.postprocessor.openvino_rerank import OpenVINORerank
reranker = OpenVINORerank(model=rerank_model_id.value, device=rerank_device.value, top_n=2)

Compiling the model to CPU ...


### Load LLM model

[back to top ⬆️](#Table-of-contents:)

OpenVINO models can be run locally through the `HuggingFacePipeline` class. To deploy a model with OpenVINO, you can specify the `backend="openvino"` parameter to trigger OpenVINO as backend inference framework.


In [27]:
available_models = []
available_models.append("INT8")


model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

Dropdown(description='Model to run:', options=('INT8',), value='INT8')

OpenVINO models can be run locally through the `OpenVINOLLM` class in [LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/llm/openvino/). If you have an Intel GPU, you can specify `device_map="gpu"` to run inference on it.

In [ ]:
from llama_index.llms.openvino import OpenVINOLLM


model_dir = int8_model_dir

print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

if "GPU" in llm_device.value and "qwen2-7b-instruct" in llm_model_id.value:
    ov_config["GPU_ENABLE_SDPA_OPTIMIZATION"] = "NO"

# On a GPU device a model is executed in FP16 precision. For red-pajama-3b-chat model there known accuracy
# issues caused by this, which we avoid by setting precision hint to "f32".
if llm_model_id.value == "red-pajama-3b-chat" and "GPU" in core.available_devices and llm_device.value in ["GPU", "AUTO"]:
    ov_config["INFERENCE_PRECISION_HINT"] = "f32"

llm = OpenVINOLLM(
    model_name=str(model_dir),
    tokenizer_name=str(model_dir),
    context_window=3900,
    max_new_tokens=2,
    model_kwargs={"ov_config": ov_config, "trust_remote_code": True},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    device_map=llm_device.value,
)

